In [3]:
import os


In [4]:
## set openai api-key
from constants import openai_key

os.environ["OPENAI_API_KEY"] = openai_key

In [5]:
from langchain.llms import  OpenAI

In [ ]:
# temperature value --> how creative we want our model to be
# 0 ---> temperature it means model is very safe. it is not taking any bets.
# 1 ---> temperature it means model is very risky. it might generate wrong output but it is very creative.

llm = OpenAI(temperature=0.6)

In [ ]:
text = "What is the capital of India"

print(llm.predict(text))

In [50]:
from langchain import HuggingFaceHub

llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature": 0, "max_length": 64})

In [ ]:
output = llm_huggingface.predict("Can you tell me the capital of Russia")
print(output)

#### Comparison of Open-source LLM and OpenAI LLM

In [ ]:
output = llm_huggingface.predict("Can you write a poem about AI")
print(output)

In [ ]:
output = llm.predict("Can you write a poem about AI")
print(output)

#### Prompt Templates and LLMChain

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'],
    template="Tell me the capital of this {country}")

prompt_template.format(country="India")

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.run("India"))

#### Combining Multiple Chains using simple Sequential Chains

In [27]:
capital_prompt = PromptTemplate(input_variables=['country'], 
                                template="Please tell me the capital of the {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_prompt)

famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[capital_chain, famous_chain])
chain.run("India")

#### Sequential Chain

In [29]:
capital_prompt = PromptTemplate(input_variables=['country'], 
                                template="Please tell me the capital of the {country}")
capital_chain = LLMChain(llm=llm, prompt=capital_prompt, output_key="capital")

famous_template = PromptTemplate(input_variables=['capital'],
                                 template="Suggest me some amazing places to visit in {capital}")
famous_chain = LLMChain(llm=llm, prompt=famous_template, output_key="places")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[capital_chain, famous_chain], 
    input_variables=['country'], 
    output_variables=['places']
)

# print(chain.run("India")
chain({'country': 'India'})

#### Chatmodels with ChatOpenAI

In [33]:
from langchain.chat_models import ChatOpenAI

In [34]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [ ]:
chat_llm = ChatOpenAI(temperature=0.6)

In [ ]:
chat_llm

In [ ]:
chat_llm.invoke([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punch lines on AI"),
])

#### Prompt Template + LLM + Output Parsers

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [42]:
class CommaSeparatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")

In [40]:
template = "You are a helpful assistant. When the user given any input, you should generate 5 words synonyms in a comma separated list."

human_template = "{text}"

chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

In [47]:
chain = chatprompt|chat_llm|CommaSeparatedOutput()

In [ ]:
chain.invoke({"text": "intelligent"})